# CDK2: Model Selection - y values shuffled
### Validation Method

In [3]:
import pandas as pd
import numpy as np
import glob, sys, os
sys.path.append('..')

In [4]:
from modules.plotting_metrics import PlotMetric
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', context='talk', font_scale=0.8)

In [5]:
file_name = './df_DkSc_results_COCRYS_CSAR_DEKOIS_DUD.pkl' # Created in 3_Calculating metrics...
X_merged_dksc = pd.read_pickle(file_name)
# Extract activity column
y_true_merged = X_merged_dksc['activity']
# Drop column from merged_dkksc
X_merged_dksc = X_merged_dksc.drop('activity', axis=1)
X_merged_dksc.shape


(3466, 402)

## Scaffold Splitting

In [6]:
#*************************************************
# Functions to compute stratify scaffold splitting
#*************************************************
sys.path.append('../2_Docking_analysis/')
from scaffold_splitter import train_test_scaffold_split

In [7]:
# Compute or load the dataframe containing the Generic Murcko Scaffolds
file = '../2_Docking_analysis/df_COCRYS_CSAR_DUD_DEKOIS_Murcko_Scaffolds_SMILES.obj'

df_scff_murcko = pd.read_pickle(file)
df_scff_murcko.shape

(3466, 3)

In [8]:
%run 4_Helper_Functions_Model_Selection_Grid_Search.ipynb

#  Hyperparameter Tunning: Grid Search

In [125]:
def randomize_y_labels(y_target, random_chi=0.1):
    '''Función para distribuir de forma aleatoria una fracción 
    chi del vector de etiquetas, de forma estratificada'''
    
    # Make a copy of the original vector
    y_copy = y.copy()
    
    # Get the number of actives inside the y_target vector
    n_actives = y_target.sum()
    random_size = np.floor(random_chi * n_actives)
    # Initialize the counters
    act_count = random_size
    inact_count = random_size
    
    # Create the randomized list of idexes
    idx_shuffled = np.random.choice(range(len(y)), len(y), replace=False)
    # iterate over idx_shuffled until act and inact counters == 0
    for l in idx_shuffled:
        if act_count > 0:
            if y_copy[l] == 1: # Is active, then change it to inactive
                y_copy[l] = 0
                act_count = act_count - 1
                continue
            if inact_count > 0: # If is inactive, change it to active
                y_copy[l] = 1
                inact_count = inact_count - 1
                continue
        else:
            break
    return(y_copy)
    

In [120]:
y1 = randomize_y_labels(y, 0)
y01 = randomize_y_labels(y, 0.1)
print(y.sum())
print(sum(y1 == y) - len(y))
print(sum(y01 == y) - len(y))
print(sum(y1))
print(sum(y01))

0.0
41.0
415
0
-82
415
415


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries: Shuffle *y* target values in the train set
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD and COCRYS are  treated as one unique library
#### Target values (y) will be shuffled

In [34]:
# Train and test over 
X = X_merged_dksc
# ***** Permutate y values *****
y = y_true_merged#.sample(frac=1, replace=False)

library = 'Merged'
scaffold_series = df_scff_murcko['scff_generic']

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [126]:
%%time
from sklearn.linear_model import LogisticRegression

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    y_rd = randomize_y_labels(y, chi)
    
    print('*'*60, '\n', 'Randomized y fraction (actives):', chi)
    print('Number of actives/inactives shuffled:', chi*y.sum())
    print('*'*60)

    estimator_name = 'LogReg'
    estimator = LogisticRegression(max_iter=300)
    hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6),
                   'penalty': ['l1', 'l2'], 
                   'solver': ['lbfgs', 'liblinear']}

    # RANDOM Train test splitting
    split_and_gs(library, library, estimator_name,
                 X, y_rd, estimator, hyperparams,
                 splitting='random', test_size=0.25, 
                 scaffold_series=None)
    
    print('\n')

************************************************************ 
 Randomized y fraction (actives): 0
Number of actives/inactives shuffled: 0
************************************************************
LogReg => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.887
- Train ROC-AUC:  	0.938
- Test ROC-AUC:   	0.892
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.672 	> median: 0.615, mean: 0.609
> Test best conf. ROC-AUC: 0.669 	> median: 0.612, mean: 0.608
****************************************************************


************************************************************ 
 Randomized y fraction (actives): 0.1
Number of actives/inactives shuffled: 41.5
*********

/home/ricci/miniconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.833
- Train ROC-AUC:  	0.899
- Test ROC-AUC:   	0.848
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.665 	> median: 0.610, mean: 0.604
> Test best conf. ROC-AUC: 0.624 	> median: 0.552, mean: 0.550
****************************************************************


************************************************************ 
 Randomized y fraction (actives): 0.2
Number of actives/inactives shuffled: 83.0
************************************************************
LogReg => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS *******

/home/ricci/miniconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.684
- Train ROC-AUC:  	0.837
- Test ROC-AUC:   	0.727
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.612 	> median: 0.571, mean: 0.569
> Test best conf. ROC-AUC: 0.652 	> median: 0.603, mean: 0.601
****************************************************************


************************************************************ 
 Randomized y fraction (actives): 0.5
Number of actives/inactives shuffled: 207.5
************************************************************
LogReg => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS ******

/home/ricci/miniconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.520
- Train ROC-AUC:  	0.847
- Test ROC-AUC:   	0.475
- Best hyperparameters {'C': 100.0, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.512 	> median: 0.483, mean: 0.483
> Test best conf. ROC-AUC: 0.550 	> median: 0.503, mean: 0.502
****************************************************************


CPU times: user 46.1 s, sys: 18 s, total: 1min 4s
Wall time: 11min 42s


In [127]:
%%time
from sklearn.linear_model import LogisticRegression

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    y_rd = randomize_y_labels(y, chi)
    
    print('*'*60, '\n', 'Randomized y fraction (actives):', chi)
    print('Number of actives/inactives shuffled:', chi*y.sum())
    print('*'*60)

    estimator_name = 'LogReg'
    estimator = LogisticRegression(max_iter=300)
    hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6),
                   'penalty': ['l1', 'l2'], 
                   'solver': ['lbfgs', 'liblinear']}

    # SCAFFOLD Train test splitting
    split_and_gs(library, library, estimator_name,
                 X, y_rd, estimator, hyperparams,
                 splitting='scaffold', test_size=0.25, 
                 scaffold_series=scaffold_series)

************************************************************ 
 Randomized y fraction (actives): 0
Number of actives/inactives shuffled: 0
************************************************************
LogReg => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.857
- Train ROC-AUC:  	0.949
- Test ROC-AUC:   	0.821
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.654 	> median: 0.602, mean: 0.596
> Test best conf. ROC-AUC: 0.746 	> median: 0.642, mean: 0.640
****************************************************************
************************************************************ 
 Randomized y fraction (actives): 0.1
Number of actives/inactives shuffled: 41.5
*********

/home/ricci/miniconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.819
- Train ROC-AUC:  	0.920
- Test ROC-AUC:   	0.730
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.639 	> median: 0.584, mean: 0.580
> Test best conf. ROC-AUC: 0.699 	> median: 0.614, mean: 0.613
****************************************************************
************************************************************ 
 Randomized y fraction (actives): 0.2
Number of actives/inactives shuffled: 83.0
************************************************************
LogReg => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS *******

/home/ricci/miniconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.596
- Train ROC-AUC:  	0.825
- Test ROC-AUC:   	0.588
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.602 	> median: 0.567, mean: 0.565
> Test best conf. ROC-AUC: 0.546 	> median: 0.491, mean: 0.492
****************************************************************
************************************************************ 
 Randomized y fraction (actives): 1
Number of actives/inactives shuffled: 415
************************************************************
LogReg => Train: Merged; Test: Merged; split: scaffold


/home/ricci/miniconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.524
- Train ROC-AUC:  	0.803
- Test ROC-AUC:   	0.526
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.548 	> median: 0.498, mean: 0.501
> Test best conf. ROC-AUC: 0.538 	> median: 0.461, mean: 0.461
****************************************************************
CPU times: user 44.3 s, sys: 15.6 s, total: 59.9 s
Wall time: 14min 38s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: X Gradient Boosting </h3>
<b>Merged Libraries</b>

In [128]:
%%time

from xgboost import XGBClassifier

chi_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1]

for chi in chi_values:
    y_rd = randomize_y_labels(y, chi)
    
    print('*'*80, '\n', 'Randomized y fraction (actives):', chi)
    print('Number of actives/inactives shuffled:', chi*y.sum())
    print('*'*80)
    
    estimator_name = 'XGB_tree'
    estimator = XGBClassifier()
    hyperparams = {'n_estimators': [200, 300],
                   'max_depth': [2, 3, 10, 20],
                   'learning_rate': [0.05, 0.1],
                   'gamma': [0.01, 0.1, 0.5, 1],
                   'alpha': [0.01, 0.1, 0.5, 1],
                   'subsample': [0.3, 0.5],
                   'colsample_bytree': [0.3, 0.5, 1]
                }

    # RANDOM Train test splitting
    split_and_gs(library, library, estimator_name,
                 X, y_rd, estimator, hyperparams,
                 splitting='random', test_size=0.25, 
                 scaffold_series=None,
                 # RandomizedGS
                 randomGS=True, n_iter=50)
    
    print('\n')

******************************************************************************** 
 Randomized y fraction (actives): 0
Number of actives/inactives shuffled: 0
********************************************************************************
XGB_tree => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.905
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.922
- Best hyperparameters {'subsample': 0.5, 'n_estimators': 300, 'max_depth': 20, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.3, 'alpha': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.672 	> median: 0.617, mean: 0.612
> Test best conf. ROC-AUC: 0.675 	> median: 0.605, mean: 0.600
****************************************************************


************************

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [20]:
%%time
from sklearn.svm import SVC

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': 1, 'gamma': 1} #{'C': np.geomspace(1e0, 1e2, 3), 'gamma': np.geomspace(1e-4, 1e0, 3)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.503
- Train ROC-AUC:  	0.000
- Test ROC-AUC:   	0.496
- Best hyperparameters {'C': 10.0, 'gamma': 1.0}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.521 	> median: 0.496, mean: 0.496
> Test best conf. ROC-AUC: 0.529 	> median: 0.487, mean: 0.488
****************************************************************
CPU times: user 23 s, sys: 48.2 ms, total: 23 s
Wall time: 4min 8s


In [ ]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin:ade0px'>GS: Linear SVM </h3>
<b>Merged Libraries</b>

In [10]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 3)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: Merged; Test: Merged; split: random


KeyboardInterrupt: 

In [23]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.771
- Train ROC-AUC:  	0.903
- Test ROC-AUC:   	0.717
- Best hyperparameters {'C': 1.584893192461111e-14}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.654 	> median: 0.602, mean: 0.596
> Test best conf. ROC-AUC: 0.746 	> median: 0.642, mean: 0.640
****************************************************************
CPU times: user 5.93 s, sys: 19.2 ms, total: 5.95 s
Wall time: 35.8 s


In [16]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.520
- Train ROC-AUC:  	0.536
- Test ROC-AUC:   	0.503
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 0.25}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.547 	> median: 0.519, mean: 0.519
> Test best conf. ROC-AUC: 0.545 	> median: 0.497, mean: 0.496
****************************************************************
CPU times: user 2.15 s, sys: 57.2 ms, total: 2.21 s
Wall time: 5.05 s


In [35]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = '1-NN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [1], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

1-NN => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.494
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.517
- Best hyperparameters {'n_neighbors': 1, 'p': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.522 	> median: 0.490, mean: 0.490
> Test best conf. ROC-AUC: 0.565 	> median: 0.521, mean: 0.520
****************************************************************
CPU times: user 3.12 s, sys: 8.41 ms, total: 3.13 s
Wall time: 6.1 s


In [17]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.516
- Train ROC-AUC:  	0.751
- Test ROC-AUC:   	0.484
- Best hyperparameters {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'n_estimators': 500}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.552 	> median: 0.516, mean: 0.516
> Test best conf. ROC-AUC: 0.543 	> median: 0.505, mean: 0.505
****************************************************************
CPU times: user 4.53 s, sys: 13.4 ms, total: 4.54 s
Wall time: 30.1 s
